In [1]:
# Cell 1 — Imports + small utilities
import os
import glob
import json
from pathlib import Path
from tqdm import tqdm

import numpy as np
import rasterio

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from terratorch import BACKBONE_REGISTRY
import torch.nn as nn

# For plotting (visualization cell)
import matplotlib.pyplot as plt


In [2]:
# Cell 2 — Paths (change these to your paths if different)
TRAIN_DATA_PATH = r"D:\TerraMind_Dataset\train"
VAL_DATA_PATH   = r"D:\TerraMind_Dataset\val"

# Where to store the mapping and outputs
LABEL_MAP_FILE = "label_mapping.json"
OUTPUT_DIR = Path("./terramind_training_output")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

print("Train path:", TRAIN_DATA_PATH)
print("Val   path:", VAL_DATA_PATH)
print("Output dir:", OUTPUT_DIR.absolute())


Train path: D:\TerraMind_Dataset\train
Val   path: D:\TerraMind_Dataset\val
Output dir: c:\Users\sathv\OneDrive\Desktop\Sathvik education\5th Semester\PE\terramind_training_output


In [3]:
# %pip install --force-reinstall "numpy<2.0" numpy==1.26.4


In [4]:
# %pip install --force-reinstall pytorch-lightning lightning


In [5]:
# %pip install --force-reinstall "numpy==1.26.4"


In [6]:
# Cell 3 — Build or load label mapping (remaps raw mask values -> 0..N-1)
def build_label_mapping_from_dirs(mask_dirs):
    """Scan provided mask directories and return sorted unique values -> mapping dict."""
    all_vals = set()
    for d in mask_dirs:
        files = glob.glob(os.path.join(d, "*.tif")) + glob.glob(os.path.join(d, "*.tiff"))
        for p in files:
            try:
                with rasterio.open(p) as src:
                    m = src.read(1)
                    all_vals.update(np.unique(m).tolist())
            except Exception as e:
                # keep scanning even if a file is broken
                print(f"Warning reading {p}: {e}")
    all_vals = sorted([int(v) for v in all_vals])
    label_map = {raw: i for i, raw in enumerate(all_vals)}
    return label_map, all_vals

# If existing mapping file present, load it; otherwise build from train masks (and val)
if os.path.exists(LABEL_MAP_FILE):
    print(f"Loading existing mapping from {LABEL_MAP_FILE}")
    with open(LABEL_MAP_FILE, "r") as f:
        label_map = json.load(f)
    # keys in JSON were strings — ensure they are ints as keys
    label_map = {int(k): int(v) for k, v in label_map.items()}
    unique_labels = sorted(label_map.keys())
else:
    print("Building label mapping from train+val masks (this may take a while)...")
    mask_dirs = [os.path.join(TRAIN_DATA_PATH, "masks"), os.path.join(VAL_DATA_PATH, "masks")]
    label_map, unique_labels = build_label_mapping_from_dirs(mask_dirs)
    with open(LABEL_MAP_FILE, "w") as f:
        json.dump({int(k): int(v) for k, v in label_map.items()}, f)
    print(f"Saved {LABEL_MAP_FILE}")

print("Total unique raw labels found:", len(unique_labels))
print("Example raw labels (first 20):", unique_labels[:20])


Loading existing mapping from label_mapping.json
Total unique raw labels found: 5888
Example raw labels (first 20): [978, 985, 986, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004]


In [7]:
# Cell 4 — Configuration (num_classes set from mapping)
CONFIG = {
    "image_size": 224,
    "batch_size": 1,
    "num_classes": len(label_map),   # 🔧 FIX: set from mapping
    "learning_rate": 1e-4,
    "max_epochs": 1,
    "precision": 32,
    "model_name": "terramind_v1_base",
    "modalities": ["S2L2A"],
    "bands": None
}

print("CONFIG:")
for k,v in CONFIG.items():
    print(f"  {k}: {v}")


CONFIG:
  image_size: 224
  batch_size: 1
  num_classes: 5888
  learning_rate: 0.0001
  max_epochs: 1
  precision: 32
  model_name: terramind_v1_base
  modalities: ['S2L2A']
  bands: None


In [8]:
# Cell 5 — Dataset with applied mapping
class TerraMindDataset(Dataset):
    def __init__(self, data_path, image_size=224, label_map=None):
        self.data_path = Path(data_path)
        self.images_path = self.data_path / "images"
        self.masks_path  = self.data_path / "masks"
        self.image_size = image_size
        self.label_map = label_map or {}

        self.samples = sorted([p.stem for p in self.images_path.glob("*.tif")] + 
                              [p.stem for p in self.images_path.glob("*.tiff")])

        print(f"✓ Found {len(self.samples)} samples in {data_path}")

    def __len__(self):
        return len(self.samples)

    def _remap_mask(self, mask_np):
        """
        🔧 FIX: remap raw mask values -> contiguous integer labels 0..N-1
        (Changed: CrossEntropyLoss requires contiguous class IDs)
        """
        # Use vectorized lookup through numpy; ensure dtype is integer
        # label_map.get ensures unmapped values return -1 (we catch that)
        get_fn = np.vectorize(lambda x: self.label_map.get(int(x), -1))
        mapped = get_fn(mask_np).astype(np.int64)
        if (mapped < 0).any():
            # find what values were unmapped (diagnostic)
            unmapped = np.unique(mask_np[mapped < 0])
            raise ValueError(f"Found unmapped mask values: {unmapped[:10]} (first 10). Update label_map.")
        return mapped

    def __getitem__(self, idx):
        sample_id = self.samples[idx]

        # ---------------- IMAGE ----------------
        img_path = self.images_path / f"{sample_id}.tif"
        if not img_path.exists():
            img_path = self.images_path / f"{sample_id}.tiff"

        with rasterio.open(str(img_path)) as src:
            img = src.read()

        # FORCE 12 BANDS FOR TERRAMIND
        if img.shape[0] > 12:
            img = img[:12]
        if img.shape[0] < 12:
            raise ValueError("Image has less than 12 bands. TerraMind S2L2A requires 12.")

        img = img.astype(np.float32)
        if img.max() > 2.0:
            img = img / 10000.0
        img = np.clip(img, 0, 1)

        img_tensor = torch.from_numpy(img).unsqueeze(0)  # shape (1, bands, H, W)
        img_tensor = F.interpolate(
            img_tensor,
            size=(self.image_size, self.image_size),
            mode="bilinear",
            align_corners=False
        ).squeeze(0)  # back to (bands, H, W)

        data = {"S2L2A": img_tensor}

        # ---------------- MASK ----------------
        mask_name = sample_id.replace("img_", "mask_")
        mask_path = self.masks_path / f"{mask_name}.tif"
        if not mask_path.exists():
            mask_path = self.masks_path / f"{mask_name}.tiff"

        with rasterio.open(str(mask_path)) as src:
            mask = src.read(1)

        # 🔧 FIX: remap raw labels -> contiguous indices BEFORE interpolation
        mask_mapped = self._remap_mask(mask)

        # Convert to tensor, interpolate with nearest neighbor, then long
        mask_t = torch.from_numpy(mask_mapped).unsqueeze(0).unsqueeze(0).float()
        mask_t = F.interpolate(mask_t, size=(self.image_size, self.image_size), mode="nearest")
        label = mask_t.squeeze(0).squeeze(0).long()  # final shape (H, W), dtype long

        return data, label


In [9]:
# Cell 6 — Create datasets & dataloaders (pass label_map)
# 🔧 FIX: pass label_map into datasets so masks are remapped.
train_dataset = TerraMindDataset(TRAIN_DATA_PATH, image_size=CONFIG["image_size"], label_map=label_map)
val_dataset   = TerraMindDataset(VAL_DATA_PATH,   image_size=CONFIG["image_size"], label_map=label_map)

# Windows + rasterio: use num_workers=0
train_loader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"], shuffle=True,
                          num_workers=0, pin_memory=False, drop_last=True)
val_loader   = DataLoader(val_dataset,   batch_size=CONFIG["batch_size"], shuffle=False,
                          num_workers=0, pin_memory=False)

print("DataLoaders ready!")
print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))

# Quick sanity check: sample a batch and inspect mapped labels
batch = next(iter(train_loader))
data_sample, labels_sample = batch
print("Data keys:", list(data_sample.keys()))
for k, v in data_sample.items():
    print(f"  {k}: {v.shape}, range=[{v.min():.3f}, {v.max():.3f}]")
print("Label shape:", labels_sample.shape)
print("Label min/max/unique-sample:", labels_sample.min().item(), labels_sample.max().item(), torch.unique(labels_sample)[:20])
print("CONFIG num_classes:", CONFIG["num_classes"])
assert labels_sample.min().item() >= 0 and labels_sample.max().item() < CONFIG["num_classes"], \
    "Mapped labels not in expected range!"


✓ Found 228 samples in D:\TerraMind_Dataset\train
✓ Found 57 samples in D:\TerraMind_Dataset\val
DataLoaders ready!
Train samples: 228
Val samples: 57
Data keys: ['S2L2A']
  S2L2A: torch.Size([1, 12, 224, 224]), range=[0.001, 0.388]
Label shape: torch.Size([1, 224, 224])
Label min/max/unique-sample: 3178 5277 tensor([3178, 3200, 3203, 3233, 3246, 3265, 3267, 3272, 3280, 3286, 3291, 3295,
        3298, 3303, 3305, 3307, 3315, 3319, 3323, 3324])
CONFIG num_classes: 5888


In [ ]:
# Cell 7 — Model definition (same as yours, with small comment)
class TerraMindFineTuner(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.backbone = BACKBONE_REGISTRY.build(
            config["model_name"],
            pretrained=True,
            modalities=config["modalities"],
            bands=config["bands"],
            merge_method="mean"
        )

        # 🔧 NOTE: decoder output channels must equal CONFIG["num_classes"]
        self.decoder = nn.Sequential(
            nn.Conv2d(768, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, config["num_classes"], kernel_size=1)
        )

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, data):
        out = self.backbone(data)

        if isinstance(out, list):
            tokens = out[0]
        else:
            tokens = out

        B, N, C = tokens.shape
        H = W = int(N ** 0.5)

        feat = tokens.transpose(1, 2).reshape(B, C, H, W)
        feat = F.interpolate(feat, size=(self.config["image_size"], self.config["image_size"]), mode="bilinear")

        return self.decoder(feat)

    def training_step(self, batch, batch_idx):
        data, y = batch
        y_hat = self.forward(data)

        # 🔧 Extra safety: check target range (will raise early with informative message)
        if y.max() >= self.config["num_classes"] or y.min() < 0:
            raise ValueError(f"Label out of range: min={y.min().item()}, max={y.max().item()} for num_classes={self.config['num_classes']}")

        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        data, y = batch
        y_hat = self.forward(data)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.config["learning_rate"])


: 

In [ ]:
# Cell 8 — Trainer, callbacks, logger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

output_dir = OUTPUT_DIR
checkpoint_callback = ModelCheckpoint(
    dirpath=output_dir / 'checkpoints',
    filename='terramind-{epoch:02d}-{val_loss:.4f}',
    save_top_k=3,
    monitor='val_loss',
    mode='min',
    save_last=True,
    verbose=True
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=8,
    mode='min',
    verbose=True
)

logger = TensorBoardLogger(
    save_dir=output_dir / 'logs',
    name='terramind_finetuning'
)

print(f"Outputs will be saved to: {output_dir.absolute()}")

print("\nInitializing model...")
model = TerraMindFineTuner(CONFIG)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model params: total={total_params:,}, trainable={trainable_params:,}")

trainer = pl.Trainer(
    max_epochs=CONFIG['max_epochs'],
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    precision=CONFIG['precision'],
    callbacks=[checkpoint_callback, early_stop_callback],
    logger=logger,
    log_every_n_steps=10,
    gradient_clip_val=1.0,
    accumulate_grad_batches=2,
    enable_progress_bar=True,
    enable_model_summary=True
)
print("Trainer initialized.")


Outputs will be saved to: c:\Users\sathv\OneDrive\Desktop\Sathvik education\5th Semester\PE\terramind_training_output

Initializing model...


In [ ]:
# Cell 9 — Training loop (with try/except for diagnostics)
print("\n" + "="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"Estimated time: ~{len(train_loader) * CONFIG['max_epochs'] // 60} minutes")
print("="*60 + "\n")

try:
    trainer.fit(
        model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )

    print("\n" + "="*60)
    print("✅ TRAINING COMPLETED!")
    print("="*60)
    print(f"\nBest checkpoint: {checkpoint_callback.best_model_path}")
    print(f"Best val loss: {checkpoint_callback.best_model_score:.4f}")

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted")
except Exception as e:
    print(f"\n❌ Training error: {e}")
    import traceback
    traceback.print_exc()



🚀 STARTING TRAINING
Estimated time: ~3 minutes




  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | TerraMindViT     | 87.3 M | train
1 | decoder  | Sequential       | 2.8 M  | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
90.1 M    Trainable params
0         Non-trainable params
90.1 M    Total params
360.553   Total estimated model params size (MB)
182       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

INFO: 
Detected KeyboardInterrupt, attempting graceful shutdown ...
INFO:lightning.pytorch.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Cell 10 — Post-train: load best model and evaluate (optional)
if checkpoint_callback.best_model_path and os.path.exists(checkpoint_callback.best_model_path):
    print("\n🔄 Loading best model...")
    best_model = TerraMindFineTuner.load_from_checkpoint(checkpoint_callback.best_model_path, config=CONFIG)
    best_model.eval()
    if torch.cuda.is_available():
        best_model = best_model.cuda()

    print("Evaluating on validation dataset...")
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            data, labels = batch
            if torch.cuda.is_available():
                data = {k: v.cuda() for k, v in data.items()}
                labels = labels.cuda()

            outputs = best_model(data)
            preds = torch.argmax(outputs, dim=1)

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    accuracy = (all_preds == all_labels).float().mean()
    print(f"Validation accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
else:
    print("No checkpoint found, skipping evaluation.")


CONFIG['num_classes'] = 5888


In [ ]:
# Cell 11 — Visualization utility (optional)
def visualize_predictions(model, dataloader, num_samples=4, output_path=OUTPUT_DIR / "predictions.png"):
    model.eval()
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, num_samples*4))
    if num_samples == 1:
        axes = axes.reshape(1, -1)

    with torch.no_grad():
        for i, (data, labels) in enumerate(dataloader):
            if i >= num_samples:
                break
            if torch.cuda.is_available():
                data = {k: v.cuda() for k, v in data.items()}
                labels = labels.cuda()

            outputs = model(data)
            preds = torch.argmax(outputs, dim=1)

            first_modality = list(data.keys())[0]
            img = data[first_modality][0].cpu().numpy()

            # RGB visualization if available
            if img.shape[0] >= 3:
                rgb = img[:3].transpose(1, 2, 0)
                rgb = np.clip((rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-8), 0, 1)
            else:
                rgb = img[0]
                rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-8)

            axes[i, 0].imshow(rgb)
            axes[i, 0].set_title(f'Input ({first_modality})')
            axes[i, 0].axis('off')

            axes[i, 1].imshow(labels[0].cpu().numpy(), cmap='tab20')
            axes[i, 1].set_title('Ground Truth')
            axes[i, 1].axis('off')

            axes[i, 2].imshow(preds[0].cpu().numpy(), cmap='tab20')
            axes[i, 2].set_title('Prediction')
            axes[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved visualization to: {output_path}")

# Use it like:
# visualize_predictions(best_model if 'best_model' in locals() else model, val_loader, num_samples=4)


In [ ]:
# Cell 12 — Save final model + config
final_dir = OUTPUT_DIR / 'final_model'
final_dir.mkdir(exist_ok=True)
if 'best_model' in globals():
    torch.save(best_model.state_dict(), final_dir / 'weights.pth')
else:
    torch.save(model.state_dict(), final_dir / 'weights.pth')

with open(final_dir / 'config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print(f"Final model saved to: {final_dir.absolute()}")


✓ Found 228 samples in D:\TerraMind_Dataset\train
✓ Found 57 samples in D:\TerraMind_Dataset\val
DataLoaders ready!
Train samples: 228
Val samples: 57


In [ ]:
# def explore_dataset(data_path):
#     print(f"\n📁 Exploring: {data_path}")
#     print("="*60)
    
#     if not os.path.exists(data_path):
#         print(f"❌ Path does not exist!")
#         return None
    
#     subdirs = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
#     print(f"\nFound {len(subdirs)} subdirectories:")
#     for subdir in subdirs:
#         subdir_path = os.path.join(data_path, subdir)
#         files = [f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]
#         print(f"  📂 {subdir}/")
#         print(f"     Files: {len(files)}")
#         if files:
#             print(f"     Examples: {files[:3]}")
#             extensions = set([os.path.splitext(f)[1] for f in files])
#             print(f"     Extensions: {extensions}")
    
#     return subdirs

# # Explore your data
# train_subdirs = explore_dataset(TRAIN_DATA_PATH)
# val_subdirs = explore_dataset(VAL_DATA_PATH)



📁 Exploring: D:\TerraMind_Dataset\train

Found 2 subdirectories:
  📂 images/
     Files: 228
     Examples: ['img_00000.tif', 'img_00001.tif', 'img_00002.tif']
     Extensions: {'.tif'}
  📂 masks/
     Files: 228
     Examples: ['mask_00000.tif', 'mask_00001.tif', 'mask_00002.tif']
     Extensions: {'.tif'}

📁 Exploring: D:\TerraMind_Dataset\val

Found 2 subdirectories:
  📂 images/
     Files: 57
     Examples: ['img_00000.tif', 'img_00001.tif', 'img_00002.tif']
     Extensions: {'.tif'}
  📂 masks/
     Files: 57
     Examples: ['mask_00000.tif', 'mask_00001.tif', 'mask_00002.tif']
     Extensions: {'.tif'}


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np
import rasterio
from pathlib import Path

class TerraMindDataset(Dataset):
    def __init__(self, data_path, image_size=224):
        self.data_path = Path(data_path)
        self.images_path = self.data_path / "images"
        self.masks_path  = self.data_path / "masks"
        self.image_size = image_size

        self.samples = sorted([
            p.stem for p in self.images_path.glob("*.tif")
        ] + [
            p.stem for p in self.images_path.glob("*.tiff")
        ])

        print(f"✓ Found {len(self.samples)} samples in {data_path}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample_id = self.samples[idx]

        # ---------------- IMAGE ----------------
        img_path = self.images_path / f"{sample_id}.tif"
        if not img_path.exists():
            img_path = self.images_path / f"{sample_id}.tiff"

        with rasterio.open(str(img_path)) as src:
            img = src.read()

        # FORCE 12 BANDS FOR TERRAMIND
        if img.shape[0] > 12:
            img = img[:12]
        if img.shape[0] < 12:
            raise ValueError("Image has less than 12 bands. TerraMind S2L2A requires 12.")

        img = img.astype(np.float32)
        if img.max() > 2.0:
            img = img / 10000.0
        img = np.clip(img, 0, 1)

        img_tensor = torch.from_numpy(img).unsqueeze(0)
        img_tensor = F.interpolate(
            img_tensor,
            size=(self.image_size, self.image_size),
            mode="bilinear",
            align_corners=False
        ).squeeze(0)

        data = {"S2L2A": img_tensor}

        # ---------------- MASK ----------------
        mask_name = sample_id.replace("img_", "mask_")
        mask_path = self.masks_path / f"{mask_name}.tif"
        if not mask_path.exists():
            mask_path = self.masks_path / f"{mask_name}.tiff"

        with rasterio.open(str(mask_path)) as src:
            mask = src.read(1)

        mask_t = torch.from_numpy(mask).unsqueeze(0).unsqueeze(0).float()
        mask_t = F.interpolate(mask_t, size=(self.image_size, self.image_size), mode="nearest")
        label = mask_t.squeeze(0).squeeze(0).long()

        return data, label


In [ ]:
TRAIN_DATA_PATH = r"D:\TerraMind_Dataset\train"
VAL_DATA_PATH   = r"D:\TerraMind_Dataset\val"

try:
    train_dataset = TerraMindDataset(
        TRAIN_DATA_PATH,
        image_size=CONFIG['image_size']
    )

    val_dataset = TerraMindDataset(
        VAL_DATA_PATH,
        image_size=CONFIG['image_size']
    )

    # Windows + rasterio MUST use num_workers=0
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=0,
        pin_memory=False,
        drop_last=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )

    print("\n✅ DataLoaders created!")
    print(f"  Train samples: {len(train_dataset)}")
    print(f"  Val samples:   {len(val_dataset)}")
    print(f"  Train batches: {len(train_loader)}")
    print(f"  Val batches:   {len(val_loader)}")

    print("\n🔍 Testing data loading...")
    test_batch = next(iter(train_loader))
    data, labels = test_batch

    print(f"  Data keys: {list(data.keys())}")
    for key, val in data.items():
        print(f"    {key}: {val.shape}, range=[{val.min():.3f}, {val.max():.3f}]")

    print(f"  Labels: {labels.shape}, unique={len(torch.unique(labels))}")
    print("✅ Data loading successful!")

except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


✓ Found 228 samples in D:\TerraMind_Dataset\train
✓ Found 57 samples in D:\TerraMind_Dataset\val

✅ DataLoaders created!
  Train samples: 228
  Val samples:   57
  Train batches: 228
  Val batches:   57

🔍 Testing data loading...
  Data keys: ['S2L2A']
    S2L2A: torch.Size([1, 12, 224, 224]), range=[0.001, 0.403]
  Labels: torch.Size([1, 224, 224]), unique=1058
✅ Data loading successful!


In [ ]:
from terratorch import BACKBONE_REGISTRY
import pytorch_lightning as pl
import torch.nn as nn

class TerraMindFineTuner(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.backbone = BACKBONE_REGISTRY.build(
            config["model_name"],
            pretrained=True,
            modalities=config["modalities"],
            bands=config["bands"],
            merge_method="mean"
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(768, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, config["num_classes"], kernel_size=1)
        )

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, data):
        out = self.backbone(data)

        if isinstance(out, list):
            tokens = out[0]
        else:
            tokens = out

        B, N, C = tokens.shape
        H = W = int(N ** 0.5)

        feat = tokens.transpose(1, 2).reshape(B, C, H, W)
        feat = F.interpolate(feat, size=(self.config["image_size"], self.config["image_size"]), mode="bilinear")

        return self.decoder(feat)

    def training_step(self, batch, batch_idx):
        data, y = batch
        y_hat = self.forward(data)

        if y.max() >= self.config["num_classes"]:
            raise ValueError(f"Label {y.max().item()} out of range num_classes={self.config['num_classes']}")

        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        data, y = batch
        y_hat = self.forward(data)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.config["learning_rate"])


In [ ]:
output_dir = Path('./terramind_training_output')
output_dir.mkdir(exist_ok=True, parents=True)

checkpoint_callback = ModelCheckpoint(
    dirpath=output_dir / 'checkpoints',
    filename='terramind-{epoch:02d}-{val_loss:.4f}',
    save_top_k=3,
    monitor='val_loss',
    mode='min',
    save_last=True,
    verbose=True
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=8,
    mode='min',
    verbose=True
)

logger = TensorBoardLogger(
    save_dir=output_dir / 'logs',
    name='terramind_finetuning'
)

print(f"✓ Outputs will be saved to: {output_dir.absolute()}")

✓ Outputs will be saved to: c:\Users\sathv\OneDrive\Desktop\Sathvik education\5th Semester\PE\terramind_training_output


In [ ]:
print("\n🔄 Initializing model...")
model = TerraMindFineTuner(CONFIG)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 Model Statistics:")
print(f"  Total params: {total_params:,}")
print(f"  Trainable params: {trainable_params:,}")
print(f"  Size: ~{total_params * 4 / 1e9:.2f} GB")

trainer = pl.Trainer(
    max_epochs=CONFIG['max_epochs'],
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    precision=CONFIG['precision'],
    callbacks=[checkpoint_callback, early_stop_callback],
    logger=logger,
    log_every_n_steps=10,
    gradient_clip_val=1.0,
    accumulate_grad_batches=2,
    enable_progress_bar=True,
    enable_model_summary=True
)

print("\n✓ Trainer initialized")


🔄 Initializing model...

📊 Model Statistics:
  Total params: 90,138,240
  Trainable params: 90,138,240
  Size: ~0.36 GB


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores



✓ Trainer initialized


In [ ]:
import numpy as np
import rasterio
from pathlib import Path

MASK_DIR = Path(r"D:\TerraMind_Dataset\train\masks")

min_label = 999999
max_label = -1

for m in MASK_DIR.glob("*.tif"):
    with rasterio.open(m) as src:
        mask = src.read(1)
        min_label = min(min_label, mask.min())
        max_label = max(max_label, mask.max())

print("📌 TRUE GLOBAL LABEL RANGE:")
print("Min:", min_label)
print("Max:", max_label)
print("→ Required num_classes =", max_label + 1)


📌 TRUE GLOBAL LABEL RANGE:
Min: 978
Max: 7698
→ Required num_classes = 7699


In [ ]:
print("\n" + "="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"Estimated time: ~{len(train_loader) * CONFIG['max_epochs'] // 60} minutes")
print("="*60 + "\n")

try:
    trainer.fit(
        model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )
    
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETED!")
    print("="*60)
    print(f"\nBest checkpoint: {checkpoint_callback.best_model_path}")
    print(f"Best val loss: {checkpoint_callback.best_model_score:.4f}")
    
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted")
except Exception as e:
    print(f"\n❌ Training error: {e}")
    import traceback
    traceback.print_exc()


🚀 STARTING TRAINING
Estimated time: ~19 minutes




  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | TerraMindViT     | 87.3 M | train
1 | decoder  | Sequential       | 2.8 M  | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
90.1 M    Trainable params
0         Non-trainable params
90.1 M    Total params
360.553   Total estimated model params size (MB)
182       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]


❌ Training error: Target 6198 is out of bounds.


Traceback (most recent call last):
  File "C:\Users\sathv\AppData\Local\Temp\ipykernel_7516\1195684400.py", line 8, in <module>
    trainer.fit(
  File "c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 560, in fit
    call._call_and_handle_interrupt(
  File "c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 598, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "c:\Users\sathv\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 1011, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "c:\Users\sathv\AppData\Local\Programs\Python\Python

In [ ]:
batch = next(iter(train_loader))
data, labels = batch

print("Label min:", labels.min().item())
print("Label max:", labels.max().item())
print("Label unique sample:", torch.unique(labels)[:50])
print("CONFIG num_classes:", CONFIG["num_classes"])


Label min: 557
Label max: 4804
Label unique sample: tensor([557, 566, 568, 573, 575, 577, 580, 582, 584, 585, 586, 587, 588, 589,
        590, 594, 595, 597, 599, 600, 601, 603, 606, 607, 608, 611, 616, 617,
        619, 620, 622, 623, 625, 628, 629, 630, 632, 633, 634, 637, 638, 639,
        640, 641, 642, 643, 644, 646, 648, 649])
CONFIG num_classes: 10


In [ ]:
# def scan_loader(loader, name):
#     all_labels = []
#     for _, y in loader:
#         all_labels.append(y)
#     all_labels = torch.cat(all_labels)

#     print(f"\n{name}:")
#     print("Min:", all_labels.min().item())
#     print("Max:", all_labels.max().item())
#     print("Unique:", len(all_labels.unique()))


# scan_loader(train_loader, "TRAIN")
# scan_loader(val_loader, "VAL")



TRAIN:
Min: 978
Max: 7698
Unique: 5859

VAL:
Min: 1155
Max: 7460
Unique: 4622


In [ ]:
print("\n🔄 Loading best model...")
best_model = TerraMindFineTuner.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    config=CONFIG
)
best_model.eval()
if torch.cuda.is_available():
    best_model = best_model.cuda()

print("✓ Best model loaded!")

# Evaluate
print("\n📊 Evaluating...")
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        data, labels = batch
        if torch.cuda.is_available():
            data = {k: v.cuda() for k, v in data.items()}
            labels = labels.cuda()
        
        outputs = best_model(data)
        preds = torch.argmax(outputs, dim=1)
        
        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

accuracy = (all_preds == all_labels).float().mean()
print(f"\n✅ Validation Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
def visualize_predictions(model, dataloader, num_samples=4):
    model.eval()
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, num_samples*4))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    with torch.no_grad():
        for i, (data, labels) in enumerate(dataloader):
            if i >= num_samples:
                break
            
            if torch.cuda.is_available():
                data = {k: v.cuda() for k, v in data.items()}
                labels = labels.cuda()
            
            outputs = model(data)
            preds = torch.argmax(outputs, dim=1)
            
            first_modality = list(data.keys())[0]
            img = data[first_modality][0].cpu().numpy()
            
            # RGB visualization
            if img.shape[0] >= 3:
                rgb = img[:3].transpose(1, 2, 0)
                rgb = np.clip((rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-8), 0, 1)
            else:
                rgb = img[0]
                rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-8)
            
            axes[i, 0].imshow(rgb)
            axes[i, 0].set_title(f'Input ({first_modality})')
            axes[i, 0].axis('off')
            
            axes[i, 1].imshow(labels[0].cpu().numpy(), cmap='tab10')
            axes[i, 1].set_title('Ground Truth')
            axes[i, 1].axis('off')
            
            axes[i, 2].imshow(preds[0].cpu().numpy(), cmap='tab10')
            axes[i, 2].set_title('Prediction')
            axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'predictions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to: {output_dir / 'predictions.png'}")

print("\n🎨 Creating visualizations...")
visualize_predictions(best_model, val_loader, num_samples=4)


In [ ]:
final_dir = output_dir / 'final_model'
final_dir.mkdir(exist_ok=True)

torch.save(best_model.state_dict(), final_dir / 'weights.pth')

with open(final_dir / 'config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print(f"\n✓ Final model saved to: {final_dir.absolute()}")

In [ ]:
print("\n" + "="*60)
print("📊 TRAINING COMPLETE!")
print("="*60)
print(f"\nConfiguration:")
print(f"  Model: {CONFIG['model_name']}")
print(f"  Modalities: {CONFIG['modalities']}")
print(f"  Bands: {CONFIG['bands']}")
print(f"  Dataset: {len(train_dataset)} train, {len(val_dataset)} val")
print(f"  Epochs: {trainer.current_epoch + 1}")
print(f"\nResults:")
print(f"  Best val loss: {checkpoint_callback.best_model_score:.4f}")
print(f"  Final accuracy: {accuracy*100:.2f}%")
print(f"\nOutputs: {output_dir.absolute()}")
print("="*60)

print("\n📈 View training logs:")
print(f"%load_ext tensorboard")
print(f"%tensorboard --logdir {output_dir / 'logs'}")